In [134]:
import numpy as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import math

In [135]:
class NaiveBayesClassifier:
    def __init__(self):
        self.classes = None
        self.means = None
        self.variances = None
        self.priors = None
        self.M_s=0
        self.Perror=0
        self.M = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features))   
        self.priors = np.zeros(n_classes)
    
        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros(n_features)
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.var(X_c, axis=0) + 1e-9

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1) + 1

    def score(self,X,labels_X):
        predict = self.predict(X)
        result = [labels_X[i] == predict[i] for i in range(len(X))]
        return np.sum(result)/len(X)

    def predict_proba(self, X):
        likelihood = np.zeros((X.shape[0], len(self.classes)))
        for i, c in enumerate(self.classes):
            class_likelihood = stats.norm.pdf(X, loc=self.means[i], scale=np.sqrt(self.variances[i]))
            class_likelihood = np.prod(class_likelihood, axis=1)
            likelihood[:, i] = class_likelihood * self.priors[i]
        evidence = np.sum(likelihood * self.priors, axis=1) + 1e-10
        return likelihood / evidence.reshape(-1, 1)

    def fit2(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features, n_features))   
        self.priors = np.zeros(n_classes)

        # 計算先驗機率
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[i] = X_c.shape[0] / X.shape[0]
            if X_c.shape[0] == 0:
                self.means[i] = np.zeros(n_features)
                self.variances[i] = np.zeros((n_features, n_features))
            else:
                self.means[i] = np.mean(X_c, axis=0)
                self.variances[i] = np.cov(X_c, rowvar=False) + 1e-6*np.eye(n_features)
    

    def Bayes_error(self):
        # initialize M matrix
        n_classes = len(self.classes)
            
        if len(self.variances.shape) == 1:
            self.variances = self.variances.reshape(-1, 1)

        M = np.zeros((n_classes, n_classes))

        # calculate M matrix entries using loops
        for i in range(n_classes):
            for j in range(i+1, n_classes):
                delta = self.means[i]-self.means[j]
                sigma = 0.5*(self.variances[i]+self.variances[j])
                sigma_inv = np.linalg.pinv(sigma)
                m_ij = 0.125 * np.dot(delta.T, np.dot(sigma_inv, delta)) + 0.5 * np.log(np.linalg.det(sigma) / np.sqrt(np.linalg.det(self.variances[i]) * np.linalg.det(self.variances[j])))
                M[i,j] = m_ij
                M[j,i] = m_ij

        s_values = np.linspace(0.1, 0.9, 20)
        
        # 找出最佳的 s 值
        opt_s = 0
        min_error = float('inf')
        
        for s in s_values:    
            Perrors = []
            for i in range(3):
                for j in range(i+1, 3):
                    alpha_ij = math.sqrt(np.linalg.det(s*self.variances[i]+(1-s)*self.variances[j])/np.sqrt(np.linalg.det(self.variances[i])*np.linalg.det(self.variances[j])))
                    Perror = 0.25 * (1 - alpha_ij) * ((self.priors[i]*math.sqrt(s) - self.priors[j]*math.sqrt(1-s))**2 + (self.priors[i]*math.sqrt(1-s) - self.priors[j]*math.sqrt(s))**2)
                    Perrors.append(Perror)
            
            # 输出结果
            print(f"For s={s}:")
            for i in range(3):
                for j in range(i+1, 3):
                    print(f"Perror({i+1},{j+1}) = {Perrors[i*2+j-3]}")
            
            # 找到更好的 s 值時更新最佳 s 值與對應的誤差率
            min_idx = np.argmin(Perrors)
            if Perrors[min_idx] < min_error:
                min_error = Perrors[min_idx]
                opt_s = s_values[min_idx // 3]
                
        print("Optimal s: {}".format(opt_s))
        print("Bayes error rate: {}".format(min_error))


In [136]:
# 指定特徵名稱
feature_names = ['label',
                 'Alcohol', 
                 'Malic acid',
                 'Ash',
                 'Alcalinity of ash' ,
                 'Magnesium',
                 'Total phenols',
                 'Flavanoids',
                 'Nonflavanoid phenols',
                 'Proanthocyanins',
                 'Color intensity',
                 'Hue',
                 'OD280/OD315 of diluted wines',
                 'Proline' ]
data=pd.read_csv("wine.data", names=feature_names)
data=data.sample(frac=1)
half = len(data) // 2
print(data.head()) # show first 5 items.
print(data.info())

     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
160      3    12.36        3.83  2.38               21.0         88  \
87       2    11.65        1.67  2.62               26.0         88   
172      3    14.16        2.51  2.48               20.0         91   
38       1    13.07        1.50  2.10               15.5         98   
9        1    13.86        1.35  2.27               16.0         98   

     Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins   
160           2.30        0.92                  0.50             1.04  \
87            1.92        1.61                  0.40             1.34   
172           1.68        0.70                  0.44             1.24   
38            2.40        2.64                  0.28             1.37   
9             2.98        3.15                  0.22             1.85   

     Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
160             7.65  0.56                          1.58      520 

In [137]:
train=data.iloc[:half]
test=data.iloc[half:]
print(type(train),train.shape)
print(type(test),test.shape)
print(train)

<class 'pandas.core.frame.DataFrame'> (89, 14)
<class 'pandas.core.frame.DataFrame'> (89, 14)
     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
160      3    12.36        3.83  2.38               21.0         88  \
87       2    11.65        1.67  2.62               26.0         88   
172      3    14.16        2.51  2.48               20.0         91   
38       1    13.07        1.50  2.10               15.5         98   
9        1    13.86        1.35  2.27               16.0         98   
..     ...      ...         ...   ...                ...        ...   
51       1    13.83        1.65  2.60               17.2         94   
134      3    12.51        1.24  2.25               17.5         85   
99       2    12.29        3.17  2.21               18.0         88   
116      2    11.82        1.47  1.99               20.8         86   
1        1    13.20        1.78  2.14               11.2        100   

     Total phenols  Flavanoids  Nonflavanoid phenols 

In [138]:
# 抽出標籤那一行
labels = train.iloc[:,0]
labels_test= test.iloc[:,0]
print(labels.head())
# 初始化標準化器
scaler = StandardScaler()
# 對特徵值進行標準化
print(train)
print(train.shape)
labels = train.iloc[:,0].values
labels_test = test.iloc[:,0].values
train = train.drop("label",axis=1)
test  = test.drop("label",axis=1)
train_features = scaler.fit_transform(train.iloc[:,:])
test_features  = scaler.transform(test.iloc[:, :])
print(labels)
print(labels.shape)
print(test_features)
print(test_features.shape)

160    3
87     2
172    3
38     1
9      1
Name: label, dtype: int64
     label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium   
160      3    12.36        3.83  2.38               21.0         88  \
87       2    11.65        1.67  2.62               26.0         88   
172      3    14.16        2.51  2.48               20.0         91   
38       1    13.07        1.50  2.10               15.5         98   
9        1    13.86        1.35  2.27               16.0         98   
..     ...      ...         ...   ...                ...        ...   
51       1    13.83        1.65  2.60               17.2         94   
134      3    12.51        1.24  2.25               17.5         85   
99       2    12.29        3.17  2.21               18.0         88   
116      2    11.82        1.47  1.99               20.8         86   
1        1    13.20        1.78  2.14               11.2        100   

     Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins   
160

Hw1結果

In [139]:
nb = NaiveBayesClassifier()
nb.fit(train_features, labels)
x1=nb.predict(test_features)
print(x1)
print(labels_test)
print(nb.score(test_features,labels_test))


[2 1 1 1 3 2 2 1 3 3 3 2 2 2 2 1 2 1 2 2 3 1 1 3 2 2 1 2 2 2 3 3 2 2 2 2 3
 2 3 1 3 3 2 1 2 1 2 3 3 2 3 2 1 3 1 2 1 3 2 1 1 1 3 1 1 1 2 1 1 1 1 2 1 3
 1 3 1 3 2 2 2 2 1 3 2 3 3 2 3]
[2 1 1 1 3 2 2 1 3 3 3 2 2 2 2 1 2 1 2 2 3 1 1 2 2 2 1 2 2 2 3 3 2 2 2 2 3
 2 3 1 3 3 2 1 2 1 2 3 3 2 3 2 1 3 1 1 1 3 2 1 1 1 3 1 1 1 2 1 1 1 1 2 1 2
 1 3 1 3 2 2 2 2 1 3 2 3 3 2 3]
0.9662921348314607


Hw2結果

In [140]:
nb.fit2(test_features, labels)
nb.Bayes_error()


For s=0.1:
Perror(1,2) = -0.008456251599105225
Perror(1,3) = -0.0031007617455811984
Perror(2,3) = -0.008456251599105225
For s=0.14210526315789473:
Perror(1,2) = -0.011471692944190945
Perror(1,3) = -0.007823020535984083
Perror(2,3) = -0.011471692944190945
For s=0.1842105263157895:
Perror(1,2) = -0.012390463925401269
Perror(1,3) = -0.010438230894327223
Perror(2,3) = -0.012390463925401269
For s=0.22631578947368422:
Perror(1,2) = -0.011813411150345942
Perror(1,3) = -0.01150263986446985
Perror(2,3) = -0.011813411150345942
For s=0.26842105263157895:
Perror(1,2) = -0.010250108208674372
Perror(1,3) = -0.011463485211286508
Perror(2,3) = -0.010250108208674372
For s=0.31052631578947365:
Perror(1,2) = -0.008152148168797932
Perror(1,3) = -0.010716241946099131
Perror(2,3) = -0.008152148168797932
For s=0.3526315789473684:
Perror(1,2) = -0.005919104910237621
Perror(1,3) = -0.009622096308393446
Perror(2,3) = -0.005919104910237621
For s=0.39473684210526316:
Perror(1,2) = -0.003895670150696391
Perror(1,3